In [14]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow import keras
from keras import metrics

### Read csv file

In [2]:
annotation_df_fp = os.path.abspath("data/annotations.csv")
annotations_df = pd.read_csv(annotation_df_fp)

### Create X, Y data

In [3]:
crop_img_loc = annotations_df["crop_img_filename"].apply(lambda x: os.path.abspath("data/crop" + "/" + x))
crop_imgs = crop_img_loc.apply(load_img).apply(img_to_array)

X_data_np = np.stack(crop_imgs)
Y_data_np = annotations_df["label"]

In [4]:
print("X_data_np.shape: ", X_data_np.shape)
print("Y_data_np.shape: ", Y_data_np.shape)

X_data_np.shape:  (1216, 100, 100, 3)
Y_data_np.shape:  (1216,)


### Split training and test data

In [5]:
# Split into train, validation, and test split of 0.6/0.2/0.2
def split_data(images, y, split=(0.6,0.2,0.2)):
    tf.random.set_seed(1234)
    np.random.seed(1234)
    shuffle = np.random.permutation(np.arange(images.shape[0]))
    images, y = images[shuffle], y[shuffle]
    
    splits = np.multiply(len(images), split).astype(int)
    X_train, X_val, X_test = np.split(images, [splits[0], splits[0]+splits[1]])
    Y_train, Y_val, Y_test = np.split(y, [splits[0], splits[0]+splits[1]])

    return X_train, Y_train, X_val, Y_val, X_test, Y_test

X_train, Y_train, X_val, Y_val, X_test, Y_test = split_data(X_data_np, Y_data_np)
print("X_train shape: ", end='')
print(X_train.shape)
print("X_val shape: ", end='')
print(X_val.shape)
print("X_test shape: ", end='')
print(X_test.shape)
print("Y_train shape: ", end='')
print(Y_train.shape)
print("Y_val shape: ", end='')
print(Y_val.shape)
print("Y_test shape: ", end='')
print(Y_test.shape)

X_train shape: (729, 100, 100, 3)
X_val shape: (243, 100, 100, 3)
X_test shape: (244, 100, 100, 3)
Y_train shape: (729,)
Y_val shape: (243,)
Y_test shape: (244,)


### Create binary classification labels

In [7]:
Y_train_binary = np.copy(Y_train)
Y_val_binary = np.copy(Y_val)
Y_test_binary = np.copy(Y_test)

# assign 
# - without_mask = 0
# - with_mask = 1
Y_train_binary[Y_train_binary == 'without_mask'] = 0.0 
Y_train_binary[Y_train_binary == 'with_mask'] = 1.0
Y_val_binary[Y_val_binary == 'without_mask'] = 0.0 
Y_val_binary[Y_val_binary == 'with_mask'] = 1.0
Y_test_binary[Y_test_binary == 'without_mask'] = 0.0
Y_test_binary[Y_test_binary == 'with_mask'] = 1.0

### Sequential model

In [9]:
# shuffle the data to achieve some kind of random sampling
np.random.seed(0) # For reproducibility

indices = np.arange(X_train.shape[0])
shuffled_indices = np.random.permutation(indices)

# Re-order training examples and corresponding labels using the randomly
# shuffled indices.
X_train = X_train[shuffled_indices]
Y_train_binary = Y_train_binary[shuffled_indices]

In [12]:
def build_sequential_model(learning_rate=0.01):
  """Build a TF logistic regression model using Keras.

  Args:
    learning_rate: The desired learning rate for SGD.

  Returns:
    model: A tf.keras model (graph).
  """
  # This is not strictly necessary, but each time you build a model, TF adds
  # new nodes (rather than overwriting), so the colab session can end up
  # storing lots of copies of the graph when you only care about the most
  # recent. Also, as there is some randomness built into training with SGD,
  # setting a random seed ensures that results are the same on each identical
  # training run.
  tf.keras.backend.clear_session()
  np.random.seed(0)
  tf.random.set_seed(0)

  # Build a model using keras.Sequential.
  model = keras.Sequential()

  # Flatten (100x100) input to a flat array
  model.add(keras.layers.Flatten())
  
  # This layer constructs the linear set of parameters for each input feature
  # (as well as a bias), and applies a sigmoid to the result. The result is
  # binary logistic regression.
  model.add(keras.layers.Dense(
      units=1,                     # output dim (for binary classification)
      use_bias=True,               # use a bias param
      activation="sigmoid"         # apply the sigmoid function!
  ))

  # Use the SGD optimizer as usual.
  optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

  # We specify the binary_crossentropy loss (equivalent to log loss).
  # Notice that we are including 'binary accuracy' as one of the metrics that we
  # ask Tensorflow to report when evaluating the model.
  model.compile(loss='binary_crossentropy', 
                optimizer=optimizer, 
                metrics=[metrics.binary_accuracy])

  return model

In [18]:
# Build the model.
sequential_model = build_sequential_model()

history = sequential_model.fit(
  x = X_train,   # our binary training examples
  y = Y_train_binary,   # corresponding binary labels
  epochs=5,             # number of passes through the training data
  batch_size=64,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history = pd.DataFrame(history.history)
display(history)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

### [Evaluation Metrics](https://towardsdatascience.com/metrics-to-evaluate-your-machine-learning-algorithm-f10ba6e38234)
- Explain what each metric does and how it is useful


1. [Classification Accuracy](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy)

    Usage with `compile() API`

    ```python
    model.compile(optimizer=..., loss=..., metrics=[tf.keras.metrics.Accuracy()])
    ```
2. [Binary Cross Entropy Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy)

    ```python
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), ...)
    ```

3. [Confusion Matrix](https://www.tensorflow.org/api_docs/python/tf/math/confusion_matrix)

    ```python
    tf.math.confusion_matrix(
        labels,
        predictions,
        num_classes=None,
        weights=None,
        dtype=tf.dtypes.int32,
        name=None
    )
    ```
    - Need to get labels and predictions from model

4. AUC

5. F1 Score

6. MSE